# Extra credit 1
Plot a neuron's ISI distribution
Review: What are interspike intervals (ISIs)? We covered this in lecture

In [ ]:
id_of_interest = Neuron_IDs[241]
one_neuron_spike_times = spike_times_all_neurons[Neuron_IDs == id_of_interest]

one_raster=raster.sel(Neuron_ID=241)

In [ ]:
isis=np.diff(one_neuron_spike_times)
x=np.arange(0, 4, .01)

plt.hist(isis, bins=x, density=True)
plt.xlabel('Bins of interspike intervals')
plt.ylabel('Count per bin');

**Questions**
1. Are there more long intervals between spikes, or short ones?
2. During periods of high neuronal activity, should there be more long intervals, or short ones?

In [ ]:
# TODO for students:
# Find the mean of ISIs in this neuron

In [ ]:
# to_remove solution
sum(isis)/len(isis)

**Questions**
1. How does this describe our ISI distribution? How would the mean ISI change for a slower neuron?
A faster neuron?

# Extra credit 2
Fit a statistial model to reproduce a neuron's ISI distribution

Prerequisites: 
1. Understand the concept of distribution
1. Play with sliders at https://distribution-explorer.github.io/continuous/gamma.html#pdf-and-cdf-plots

Questions:
1. Look at the PDF plot (left). Does it resemble the ISI distribution at any slider positions? What are they?
1. What does changing gamma distribution's mean (alpha, a.k.a. shape) do?
1. What about its variance (beta, a.k.a. scale)?


In [ ]:
# @title Maintenance
# Version for Colab:
!pip install -q numpyro@git+https://github.com/pyro-ppl/numpyro arviz
# Version for Python on you own computer:
# !pip install numpyro jax jaxlib arviz scipy
from jax import random
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import arviz as az
import numpyro
import scipy.stats as stats

In [ ]:
def model(isis):
    # Our initial broad guesses at scale and shape:
    scale = numpyro.sample('scale', dist.Gamma(1., 4))
    shape = numpyro.sample('shape', dist.Gamma(1., 4))
    # Provide obvserved data:
    numpyro.sample('obs', dist.Gamma(scale, shape), obs=isis)


# Try to reproduce ISIs by drawing some samples from our model.
# It will arrive at an estimate of scale and shape
mcmc = MCMC(NUTS(model), 1000, 2000)
mcmc.run(random.PRNGKey(0),isis)

# Convert to dataarray
data = az.from_numpyro(mcmc)
# Take a mean
best_guess=data.posterior.mean(['chain','draw'])

In [ ]:
best_guess

In [ ]:
isi_fit=stats.gamma.pdf(x=x,
                    a=best_guess['scale'],
                    scale=best_guess['shape'])

plt.plot(x, isi_fit)
plt.hist(isis, bins=x, density=True);
